In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
airline_codes_file = "Airline Codes.csv"
airline_codes_df = pd.read_csv(airline_codes_file)
airline_codes_df.head()

,Code,Description
0,19031,Mackey International Inc.
1,19032,Munz Northern Airlines Inc.
2,19033,Cochise Airlines Inc.
3,19034,Golden Gate Airlines Inc.
4,19035,Aeromech Inc.


In [3]:
# Create a filtered dataframe from specific columns
airline_codes_cols = ["Code", "Description"]
airline_codes_transformed= airline_codes_df[airline_codes_cols].copy()

# Rename the column headers
airline_codes_transformed = airline_codes_df.rename(columns={"Code": "airline_id",
                                                            "Description": "airline_name"
                                                            })

# Clean the data by dropping duplicates and setting the index
airline_codes_transformed.drop_duplicates("airline_id", inplace=True)
airline_codes_transformed.set_index("airline_id", inplace=True)

airline_codes_transformed.head()

,airline_name
airline_id,
19031,Mackey International Inc.
19032,Munz Northern Airlines Inc.
19033,Cochise Airlines Inc.
19034,Golden Gate Airlines Inc.
19035,Aeromech Inc.


In [4]:
flight_data_file = "Jan_2020_ontime 2.csv"
flight_data_df = pd.read_csv(flight_data_file)
flight_data_df.head()

,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,...,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE,Unnamed: 21
0,1,3,EV,20366,EV,N48901,4397,13930,1393007,ORD,...,GRB,1003.0,0.0,1000-1059,1117.0,0.0,0.0,0.0,174.0,NaN
1,1,3,EV,20366,EV,N16976,4401,15370,1537002,TUL,...,ORD,1027.0,0.0,1000-1059,1216.0,0.0,0.0,0.0,585.0,NaN
2,1,3,EV,20366,EV,N12167,4404,11618,1161802,EWR,...,TYS,1848.0,0.0,1800-1859,2120.0,0.0,0.0,0.0,631.0,NaN
3,1,3,EV,20366,EV,N14902,4405,10781,1078105,BTR,...,IAH,1846.0,0.0,1800-1859,2004.0,0.0,0.0,0.0,253.0,NaN
4,1,3,EV,20366,EV,N606UX,4407,14524,1452401,RIC,...,IAH,1038.0,0.0,1000-1059,1330.0,0.0,0.0,0.0,1157.0,NaN


In [6]:
# Create a filtered dataframe from specific columns
flight_data_cols = ["OP_CARRIER_AIRLINE_ID", "OP_CARRIER_FL_NUM", "ORIGIN", "DEST", "DEP_TIME", "DEP_DEL15", "DAY_OF_MONTH", "DAY_OF_WEEK"]
flight_data_transformed= flight_data_df[flight_data_cols].copy()

# Rename the column headers
flight_data_transformed = flight_data_transformed.rename(columns={"OP_CARRIER_AIRLINE_ID": "airline_id",
                                                                 "OP_CARRIER_FL_NUM": "flight_number",
                                                                 "ORIGIN": "departure_airport",
                                                                 "DEST": "destination_airport",
                                                                 "DEP_TIME": "departure_time",
                                                                 "DEP_DEL15": "departure_delay_15_mins",
                                                                 "DAY_OF_MONTH": "day_of_the_month",
                                                                 "DAY_OF_WEEK":"day_of_the_week"
                                                                 })

#Add a flight_id column and set it as the index
flight_data_transformed.insert(0, 'flight_id', range(0, 0 + len(flight_data_transformed)))
flight_data_transformed.set_index("flight_id", inplace=True)

flight_data_transformed.head()

,airline_id,flight_number,departure_airport,destination_airport,departure_time,departure_delay_15_mins,day_of_the_month,day_of_the_week
flight_id,,,,,,,,
0,20366,4397,ORD,GRB,1003.0,0.0,1,3
1,20366,4401,TUL,ORD,1027.0,0.0,1,3
2,20366,4404,EWR,TYS,1848.0,0.0,1,3
3,20366,4405,BTR,IAH,1846.0,0.0,1,3
4,20366,4407,RIC,IAH,1038.0,0.0,1,3


In [7]:
connection_string = "postgres:postgres@localhost:5432/Jan_2020_flight_data_db"
engine = create_engine(f'postgresql://{connection_string}')

In [8]:
engine.table_names()

['airline_codes', 'flight_datas']

In [ ]:
flight_data_transformed.to_sql(name='flight_datas', con=engine, if_exists='append', index=True)

In [ ]:
airline_codes_transformed.to_sql(name='airline_codes', con=engine, if_exists='append', index=True)